# Initialisation

In [15]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.style.use('seaborn')

import ta
import plotly
import plotly.express as px
import plotly.graph_objects as go
import cufflinks as cf

from datetime import datetime

In [3]:
from scipy.stats import linregress

In [33]:
# Load the raw data
df_l = pd.read_csv('./FX Data/GBPUSD.csv', names=['date','open','high','low','close'])

In [34]:
df = df_l[0:500]

In [35]:
len(df_l)


5180

# Charts

In [36]:
#fig_cdls = go.Figure(data=[go.Candlestick(x=df['Date'][0:100],open=df['Open'],high=df['High'],low=df['Low'],close=df['Close'])])
#fig_cdls.show()

In [37]:
#fig = px.line(df[0:500],x='Date',y='Close')
#fig.show()

In [38]:
# ind_EMA = ta.trend.EMAIndicator(close=df['Close'],n=10,fillna=False)

# df['EMA'] = ind_EMA.ema_indicator()

In [39]:
# fig_ind = go.Figure()

# fig_ind.update_layout(xaxis_range=[0,50],
#                       yaxis_range=(1.5,1.7),
#                  title_text = "Close and EMA")

# fig_ind.add_trace(go.Scatter(
#                     x=df['Date'],
#                     y=df['Close'],
#                     name='Close',
#                     line_color='dimgray',
#                     opacity=0.4))

# fig_ind.add_trace(go.Scatter(
#                     x=df['Date'],
#                     y=df['EMA'],
#                     name='EMA',
#                     line_color='deepskyblue',
#                     opacity=0.4))

# fig_ind.show()

# Adding features
## Technical Analysis features

In [40]:
df['wclose'] = (df['close']*2+df['high']+df['low'])/4

In [41]:
for i in range(1,len(df)):
    if df.at[i,'high'] > df.at[i-1,'high']:
        df.at[i,'delta'] = 1
    else:
        df.at[i,'delta'] = 0

In [42]:
def add_SO(i):
    ind_SO = ta.momentum.StochasticOscillator(high=df['high'],low=df['low'],close=df['close'],n=i)
    name = 'so_%s' %(i)
    df[name] = ind_SO.stoch()
    return

for i in [3,4,5,8,9,10]:
    add_SO(i)

In [43]:
def add_WR(i):
    ind_WR = ta.momentum.WilliamsRIndicator(high=df['high'],low=df['low'],close=df['close'],lbp=i)
    name = 'wr_%s' %(i)
    df[name] = ind_WR.wr()
    return

for i in [6,7,8,9,10]:
    add_WR(i)

In [44]:
def add_ROC(i):
    ind_ROC = ta.momentum.ROCIndicator(close=df['close'],n=i)
    name = 'roc_%s' %(i)
    df[name] = ind_ROC.roc()
    return

for i in [12,13,14,15]:
    add_ROC(i)

In [45]:
def add_WCP(i):
    ind_WCP = ta.trend.EMAIndicator(close=df['wclose'],n=i)
    name = 'wcp_%s' %(i)
    df[name] = ind_WCP.ema_indicator()
    return

add_WCP(15)

In [46]:
def add_MACD(i,j):
    ind_MACD = ta.trend.MACD(close=df['close'],n_fast=i,n_slow=j)
    name ='macd_%s_%s' %(i,j)
    df[name] = ind_MACD.macd()
    return

add_MACD(15,30)  

In [47]:
def add_CCI(i):
    ind_CCI = ta.trend.cci(high=df['high'],low=df['low'],close=df['close'],n=i)
    name = 'cci_%s' %(i)
    df[name] = ind_CCI
    return

add_CCI(15)

## Signal Processing features

In [48]:
for i in range(1,len(df)):
    df.at[i,'hi_avg_2'] =  (df.at[i-1,'high'] + df.at[i,'high'])/2
    df.at[i,'lo_avg_2'] =  (df.at[i-1,'low'] + df.at[i,'low'])/2
    df.at[i,'hilo_avg_2'] = (df.at[i,'hi_avg_2'] + df.at[i,'lo_avg_2'])/2
    df.at[i,'hilo_avg'] = (df.at[i,'high'] + df.at[i,'low'])/2

In [49]:
def add_slope(i):
    for j in range(i-1,len(df)):
        a=df['high'][j-(i-1):j+1]
        b=[]
        for k in range(i):
            b.append(k)
        name = 'slope_%s' %(i)
        df.at[j,name] = linregress(a,b).slope
    
for i in [3,4,5,10,20,30]:
    add_slope(i)

In [52]:
df = df[30:]

In [53]:
df.head(n=10)

,date,open,high,low,close,wclose,delta,so_3,so_4,so_5,...,hi_avg_2,lo_avg_2,hilo_avg_2,hilo_avg,slope_3,slope_4,slope_5,slope_10,slope_20,slope_30
30,14/2/2000,1.5927,1.5971,1.5860,1.5873,1.589425,0.0,4.166667,4.153355,4.153355,...,1.60375,1.58695,1.595350,1.59155,-96.150780,-124.492058,-143.325143,-106.453867,-249.893985,-366.236733
31,15/2/2000,1.5873,1.5976,1.5851,1.5960,1.593675,1.0,43.083004,33.956386,33.850932,...,1.59735,1.58555,1.591450,1.59135,-112.616576,-122.682684,-147.339957,-117.987932,-248.908844,-363.849731
32,16/2/2000,1.5960,1.6100,1.5946,1.6072,1.604750,1.0,88.755020,87.351779,68.847352,...,1.60380,1.58985,1.596825,1.60230,120.778978,-2.117010,-87.880276,-35.636482,-248.159165,-365.522179
33,17/2/2000,1.6072,1.6138,1.6027,1.6083,1.608275,1.0,80.836237,80.836237,80.836237,...,1.61190,1.59865,1.605275,1.60825,112.855285,142.663121,80.169944,27.926070,-250.532839,-365.163995
34,18/2/2000,1.6083,1.6089,1.5955,1.5995,1.600850,0.0,25.520833,50.174216,50.174216,...,1.61135,1.59910,1.605225,1.60220,-83.207262,129.209151,170.327131,-18.140085,-259.440605,-365.166326
35,21/2/2000,1.5995,1.6040,1.5978,1.6003,1.600600,0.0,26.229508,29.687500,52.961672,...,1.60645,1.59665,1.601550,1.60090,-204.081633,-234.019723,74.946192,-169.733823,-275.071713,-367.044806
36,22/2/2000,1.6003,1.6193,1.6002,1.6136,1.611675,1.0,76.050420,76.050420,76.923077,...,1.61165,1.59900,1.605325,1.60975,85.185387,44.912498,67.206354,-17.270112,-269.368690,-362.290756
37,23/2/2000,1.6136,1.6146,1.6020,1.6042,1.606250,0.0,29.767442,36.554622,36.554622,...,1.61695,1.60110,1.609025,1.60830,86.286428,121.530383,87.701348,105.522302,-252.827907,-360.123812
38,24/2/2000,1.6042,1.6060,1.5976,1.5991,1.600450,0.0,6.912442,6.912442,15.126050,...,1.61030,1.59980,1.605050,1.60180,-146.185975,4.170744,30.133340,182.303669,-237.809460,-360.585061
39,25/2/2000,1.5991,1.6005,1.5887,1.5931,1.593850,0.0,16.988417,14.379085,14.379085,...,1.60325,1.59315,1.598200,1.59460,-139.594746,-151.968578,-83.323756,122.482201,-226.218786,-360.949545


# Classifiers

<img src="files/algorithm cheat sheet.png">

In [54]:
from sklearn.svm import SVC

## RBF Classifier

In [61]:
training_labels =df['close']

In [62]:
type(training_labels)

pandas.core.series.Series

In [63]:
feature_matrix=df.drop(columns=['date','open','high','low','close','wclose','delta'])

In [66]:
feature_matrix

,so_3,so_4,so_5,so_8,so_9,so_10,wr_6,wr_7,wr_8,wr_9,...,hi_avg_2,lo_avg_2,hilo_avg_2,hilo_avg,slope_3,slope_4,slope_5,slope_10,slope_20,slope_30
30,4.166667,4.153355,4.153355,11.242604,11.242604,11.111111,-95.846645,-88.757396,-88.757396,-88.757396,...,1.60375,1.58695,1.595350,1.59155,-96.150780,-1.244921e+02,-143.325143,-106.453867,-249.893985,-366.236733
31,43.083004,33.956386,33.850932,36.982249,36.982249,36.982249,-66.149068,-66.149068,-63.017751,-63.017751,...,1.59735,1.58555,1.591450,1.59135,-112.616576,-1.226827e+02,-147.339957,-117.987932,-248.908844,-363.849731
32,88.755020,87.351779,68.847352,68.633540,70.118343,70.118343,-31.366460,-31.366460,-31.366460,-29.881657,...,1.60380,1.58985,1.596825,1.60230,120.778978,-2.117010e+00,-87.880276,-35.636482,-248.159165,-365.522179
33,80.836237,80.836237,80.836237,72.049689,72.049689,73.372781,-27.725857,-27.950311,-27.950311,-27.950311,...,1.61190,1.59865,1.605275,1.60825,112.855285,1.426631e+02,80.169944,27.926070,-250.532839,-365.163995
34,25.520833,50.174216,50.174216,44.720497,44.720497,44.720497,-49.825784,-55.140187,-55.279503,-55.279503,...,1.61135,1.59910,1.605225,1.60220,-83.207262,1.292092e+02,170.327131,-18.140085,-259.440605,-365.166326
35,26.229508,29.687500,52.961672,47.352025,47.204969,47.204969,-47.038328,-47.038328,-52.647975,-52.795031,...,1.60645,1.59665,1.601550,1.60090,-204.081633,-2.340197e+02,74.946192,-169.733823,-275.071713,-367.044806
36,76.050420,76.050420,76.923077,83.333333,83.333333,83.333333,-16.666667,-16.666667,-16.666667,-16.666667,...,1.61165,1.59900,1.605325,1.60975,85.185387,4.491250e+01,67.206354,-17.270112,-269.368690,-362.290756
37,29.767442,36.554622,36.554622,55.847953,55.847953,55.847953,-61.133603,-44.152047,-44.152047,-44.152047,...,1.61695,1.60110,1.609025,1.60830,86.286428,1.215304e+02,87.701348,105.522302,-252.827907,-360.123812
38,6.912442,6.912442,15.126050,40.935673,40.935673,40.935673,-84.873950,-81.781377,-59.064327,-59.064327,...,1.61030,1.59980,1.605050,1.60180,-146.185975,4.170744e+00,30.133340,182.303669,-237.809460,-360.585061
39,16.988417,14.379085,14.379085,14.379085,23.391813,23.391813,-85.620915,-85.620915,-85.620915,-76.608187,...,1.60325,1.59315,1.598200,1.59460,-139.594746,-1.519686e+02,-83.323756,122.482201,-226.218786,-360.949545
